# Caso 3

In [1]:
!pip install geopy


In [3]:
!pip install pandas

  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- --------

In [2]:
from geopy.distance import geodesic
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory


ModuleNotFoundError: No module named 'pandas'

# Formulación matemática del problema – Caso 3

## 1. Formulación del Modelo

En este caso se parte del modelo original del problema de ruteo con capacidad (CVRP) implementado en el caso base, el cual fue ampliado con las siguientes consideraciones adicionales:

1. Inclusión de estaciones de recarga, con costos por litro de combustible.
2. Consideración de la autonomía máxima por vehículo.
3. Costo de peajes por tonelada según la localidad.
4. Restricción de peso máximo permitido por municipio.

A continuación se detallan los conjuntos, parámetros, variables, función objetivo y restricciones del modelo.

### 1.1. Conjuntos

* \( L = \{1, ..., n\} \): Conjunto de localidades (1: puerto, 2–15: municipios, 16+: estaciones).
* \( D = \{2, ..., 15\} \): Conjunto de municipios con demanda.
* \( E \subseteq L \): Conjunto de estaciones de recarga.
* \( V = \{1, ..., m\} \): Conjunto de vehículos disponibles.

### 1.2. Índices

* \( i, j \in L \): Localidades origen y destino.
* \( k \in V \): Vehículo.

### 1.3. Parámetros

* \( distancias_{ij} \): Distancia entre la localidad \( i \) y la localidad \( j \).
* \( D\_demanda_i \): Demanda del municipio \( i \in D \).
* \( V\_capacidad_k \): Capacidad máxima de carga del vehículo \( k \).
* \( V\_autonomia_k \): Autonomía máxima en kilómetros del vehículo \( k \).
* \( precio\_combustible_j \): Precio por litro de combustible en la estación \( j \in E \).
* \( peso\_max_i \): Peso máximo permitido en el municipio \( i \in D \).
* \( peaje_i \): Costo por tonelada si el vehículo pasa por la localidad \( i \) (0 si no aplica).

### 1.4. Variables de decisión

* \( x_{ijk} \in \{0,1\} \): Toma valor 1 si el vehículo \( k \) viaja de \( i \) a \( j \), 0 en otro caso.
* \( u_{ik} \in \mathbb{Z} \): Orden de visita del nodo \( i \) por el vehículo \( k \) (usado para eliminar subciclos).
* \( r_{jk} \geq 0 \): Litros de combustible recargados por el vehículo \( k \) en \( j \).
* \( f_{jk} \geq 0 \): Combustible restante del vehículo \( k \) al llegar a la localidad \( j \).
* \( w_{ik} \geq 0 \): Peso que transporta el vehículo \( k \) al llegar a la localidad \( i \).

---

## 2. Función Objetivo

Minimizar el costo total de transporte, que incluye: distancia recorrida, combustible recargado y peajes:

$$
\min \sum_{k \in V} \sum_{\substack{i,j \in L \\ i \ne j}} distancias_{ij} \cdot x_{ijk} 
+ \sum_{k \in V} \sum_{j \in E} precio\_combustible_j \cdot r_{jk} 
+ \sum_{k \in V} \sum_{i \in D} peaje_i \cdot w_{ik}
$$

---

## 3. Restricciones

### (0) Prohibir viajes de un nodo a sí mismo:

$$
x_{iik} = 0 \quad \forall i \in L, \forall k \in V
$$

### (1) Cada municipio debe ser visitado exactamente una vez:

$$
\sum_{k \in V} \sum_{\substack{i \in L \\ i \ne j}} x_{ijk} = 1 \quad \forall j \in D
$$

### (2) Cada vehículo debe salir una sola vez del puerto:

$$
\sum_{j \in L, j \ne 1} x_{1jk} = 1 \quad \forall k \in V
$$

### (3) Cada vehículo debe regresar una vez al puerto:

$$
\sum_{i \in L, i \ne 1} x_{i1k} = 1 \quad \forall k \in V
$$

### (4) Conservación de flujo en los municipios:

$$
\sum_{\substack{i \in L \\ i \ne h}} x_{ihk} = \sum_{\substack{j \in L \\ j \ne h}} x_{hjk} \quad \forall h \in D, \forall k \in V
$$

### (5) Eliminación de subciclos (MTZ):

$$
u_{ik} - u_{jk} + n \cdot x_{ijk} \leq n - 1 \quad \forall i \ne j \in D, \forall k \in V
$$

### (6) Restricción de capacidad por vehículo:

$$
\sum_{i \in D} D\_demanda_i \cdot \sum_{\substack{j \in L \\ j \ne i}} x_{ijk} \leq V\_capacidad_k \quad \forall k \in V
$$

### (7) Restricción de autonomía por vehículo:

$$
\sum_{\substack{i,j \in L \\ i \ne j}} distancias_{ij} \cdot x_{ijk} \leq V\_autonomia_k + \sum_{j \in E} r_{jk} \quad \forall k \in V
$$

### (8) Restricción de peso máximo por municipio:

$$
w_{ik} \leq peso\_max_i \quad \forall i \in D, \forall k \in V
$$

---



In [17]:
import pandas as pd
from geopy.distance import geodesic

# -------------------------------
# 1. CARGA DE ARCHIVOS DE CASO 3
# -------------------------------
clientes = pd.read_csv('Proyecto_C_Caso3/clients.csv')
depositos = pd.read_csv('Proyecto_C_Caso3/depots.csv')
estaciones = pd.read_csv('Proyecto_C_Caso3/stations.csv')
vehiculos = pd.read_csv('Proyecto_C_Caso3/vehicles.csv')
peajes = pd.read_csv('Proyecto_C_Caso3/tolls.csv')

# -------------------------------
# 2. CREACIÓN DEL locations.csv UNIFICADO
# -------------------------------
# Empezar con los 15 nodos originales del caso base
df_base = pd.read_csv('Proyecto_Caso_Base/locations.csv')
df_base = df_base[df_base['LocationID'] < 16]  # Solo del 1 al 15
df_base.to_csv('Proyecto_C_Caso3/locations.csv', index=False)

# Añadir estaciones al archivo de localidades
for i in range(len(estaciones)):
    data = {
        'LocationID': estaciones.loc[i, 'LocationID'],
        'Longitude': estaciones.loc[i, 'Longitude'],
        'Latitude': estaciones.loc[i, 'Latitude']
    }
    pd.DataFrame([data]).to_csv('Proyecto_C_Caso3/locations.csv', mode='a', header=False, index=False)

# Leer el archivo combinado de localidades
locations_csv = pd.read_csv('Proyecto_C_Caso3/locations.csv')

# -------------------------------
# 3. MATRIZ DE DISTANCIAS ENTRE NODOS
# -------------------------------
distancias = []
for i in range(len(locations_csv)):
    coord_i = (locations_csv.loc[i, 'Latitude'], locations_csv.loc[i, 'Longitude'])
    fila = []
    for j in range(len(locations_csv)):
        coord_j = (locations_csv.loc[j, 'Latitude'], locations_csv.loc[j, 'Longitude'])
        distancia = geodesic(coord_i, coord_j).kilometers
        fila.append(distancia)
    distancias.append(fila)

# -------------------------------
# 4. DEFINICIÓN DE CONJUNTOS
# -------------------------------
num_localidades = len(locations_csv)       # Total: 27
num_vehiculos = len(vehiculos)             # Según vehicles.csv

L = list(range(1, num_localidades + 1))    # Todas las localidades
D = list(range(2, 16))                     # Municipios con demanda
E = list(estaciones['LocationID'])         # Estaciones de recarga
V = list(range(1, num_vehiculos + 1))      # Vehículos disponibles

# -------------------------------
# 5. PARÁMETROS: demanda, capacidad, autonomía
# -------------------------------
D_demanda = {
    row['LocationID']: row['Demand']
    for _, row in clientes.iterrows()
}

V_capacidad = {
    i + 1: vehiculos.loc[i, 'Capacity']
    for i in range(num_vehiculos)
}

V_autonomia = {
    i + 1: vehiculos.loc[i, 'Range']
    for i in range(num_vehiculos)
}

# -------------------------------
# 6. PRECIO DE COMBUSTIBLE EN ESTACIONES
# -------------------------------
precio_combustible = {
    int(row['LocationID']): float(row['FuelCost'])
    for _, row in estaciones.iterrows()
}


# -------------------------------
# 7. PESO MÁXIMO PERMITIDO POR MUNICIPIO
# -------------------------------
# Reemplazar NaN por un valor alto por defecto si es necesario
clientes['MaxWeight'] = clientes['MaxWeight'].fillna(9999)

peso_max = {
    row['LocationID']: row['MaxWeight']
    for _, row in clientes.iterrows()
}

# -------------------------------
# 8. COSTO DE PEAJE POR MUNICIPIO
# -------------------------------
# Dado que TollName no tiene (i, j), se asocia el costo por nodo usando el índice
# (asumimos que cada peaje se asocia secuencialmente a D: municipios 2–15)

# Limpieza al construir el diccionario
costo_peaje = {}
for idx, row in peajes.iterrows():
    try:
        loc_id = D[idx]
        costo = row['RatePerTon']
        costo_peaje[loc_id] = 0 if pd.isna(costo) else float(costo)
    except IndexError:
        continue


# -------------------------------
# 9. VERIFICACIÓN
# -------------------------------
print(f"Total de localidades cargadas: {len(locations_csv)}")
print("Distancia entre nodo 1 y nodo 2:", distancias[0][1])
print("Ejemplo demanda:", D_demanda)
print("Precio combustible estaciones:", precio_combustible)
print("Peso máximo por municipio:", peso_max)
print("Costo de peaje por municipio:", costo_peaje)


Total de localidades cargadas: 27
Distancia entre nodo 1 y nodo 2: 17.182366983108864
Ejemplo demanda: {2: 16.0, 3: 18.0, 4: 16.0, 5: 18.0, 6: 15.0, 7: 17.6, 8: 17.6, 9: 10.0, 10: 11.0, 11: 9.0, 12: 5.0, 13: 7.0, 14: 5.0, 15: 10.0}
Precio combustible estaciones: {16: 13500.0, 17: 14000.0, 18: 13800.0, 19: 15200.0, 20: 14800.0, 21: 15500.0, 22: 16000.0, 23: 16500.0, 24: 16200.0, 25: 15800.0, 26: 15300.0, 27: 14900.0}
Peso máximo por municipio: {2: 9999.0, 3: 9999.0, 4: 35.0, 5: 9999.0, 6: 35.0, 7: 9999.0, 8: 25.0, 9: 50.0, 10: 40.0, 11: 38.0, 12: 10.0, 13: 50.0, 14: 10.0, 15: 9999.0}
Costo de peaje por municipio: {2: 800.0, 3: 700.0, 4: 650.0, 5: 600.0, 6: 0, 7: 750.0, 8: 550.0, 9: 0, 10: 0, 11: 550.0, 12: 0, 13: 300.0, 14: 0, 15: 4000.0}


# Resolución Caso 3|

In [22]:
!pip install glpk

from pyomo.environ import SolverFactory
SolverFactory('glpk').available()


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build glpk


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [24 lines of output]
  C:\Users\Mariana\AppData\Local\Temp\pip-build-env-n5vi1ic7\overlay\Lib\site-packages\setuptools\dist.py:761: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved :: GNU General Public License (GPL)
  
          See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
          ********************************************************************************
  
  !!
    self._finalize_license_expression()
  running bdist_wheel
  running build
  running build_ext
  building 'glpk' extension
  creating build\temp.win-amd64-cpython-39\Release\src
  "C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.39.33

False

In [19]:
from pyomo.environ import *

# -------------------------
# CREACIÓN DEL MODELO
# -------------------------
model = ConcreteModel()

# -------------------------
# CONJUNTOS
# -------------------------
model.L = Set(initialize=L)       # Localidades
model.D = Set(initialize=D)       # Municipios con demanda
model.E = Set(initialize=E)       # Estaciones de recarga
model.V = Set(initialize=V)       # Vehículos

# -------------------------
# VARIABLES DE DECISIÓN
# -------------------------
model.x = Var(model.L, model.L, model.V, domain=Binary)              # Ruta del vehículo
model.u = Var(model.L, model.V, domain=NonNegativeIntegers)          # Orden de visita (MTZ)
model.r = Var(model.L, model.V, domain=NonNegativeReals)            # Litros recargados
model.f = Var(model.L, model.V, domain=NonNegativeReals)            # Combustible restante
model.w = Var(model.L, model.V, domain=NonNegativeReals)            # Peso transportado

# -------------------------
# PARÁMETROS
# -------------------------
def dist_rule(model, i, j):
    return distancias[i-1][j-1]


model.dist = Param(model.L, model.L, initialize=dist_rule, within=NonNegativeReals, mutable=True)
model.demanda = Param(model.D, initialize=D_demanda, within=NonNegativeReals)
model.capacidad = Param(model.V, initialize=V_capacidad, within=NonNegativeReals)
model.autonomia = Param(model.V, initialize=V_autonomia, within=NonNegativeReals)
model.precio_comb = Param(model.L, initialize=precio_combustible, within=NonNegativeReals, default=0)
model.peso_max = Param(model.D, initialize=peso_max, within=NonNegativeReals, default=9999)
model.peaje = Param(model.D, initialize=costo_peaje, within=NonNegativeReals, default=0)

# -------------------------
# FUNCIÓN OBJETIVO
# -------------------------
def obj_rule(model):
    return (
        sum(model.dist[i, j] * model.x[i, j, k] for i in model.L for j in model.L if i != j for k in model.V) +
        sum(model.precio_comb[j] * model.r[j, k] for j in model.E for k in model.V) +
        sum(model.peaje[i] * model.w[i, k] for i in model.D for k in model.V)
    )

model.obj = Objective(rule=obj_rule, sense=minimize)

# -------------------------
# RESTRICCIONES
# -------------------------
model.res = ConstraintList()

# (0) Prohibir viajes del mismo nodo a sí mismo
for i in L:
    for k in V:
        model.res.add(model.x[i, i, k] == 0)

# (1) Cada cliente debe ser visitado una sola vez
for j in D:
    model.res.add(sum(model.x[i, j, k] for i in L if i != j for k in V) == 1)

# (2) Cada vehículo debe salir del puerto una vez
for k in V:
    model.res.add(sum(model.x[1, j, k] for j in L if j != 1) == 1)

# (3) Cada vehículo debe regresar al puerto una vez
for k in V:
    model.res.add(sum(model.x[i, 1, k] for i in L if i != 1) == 1)

# (4) Conservación de flujo
for h in D:
    for k in V:
        model.res.add(
            sum(model.x[i, h, k] for i in L if i != h) ==
            sum(model.x[h, j, k] for j in L if j != h)
        )

# (5) Subtour elimination (MTZ)
n = len(L)
for i in D:
    for j in D:
        if i != j:
            for k in V:
                model.res.add(model.u[i, k] - model.u[j, k] + n * model.x[i, j, k] <= n - 1)

# (6) Capacidad máxima por vehículo
for k in V:
    model.res.add(
        sum(model.demanda[i] * sum(model.x[i, j, k] for j in L if j != i) for i in D)
        <= model.capacidad[k]
    )

# (7) Autonomía más recarga
for k in V:
    model.res.add(
        sum(model.dist[i, j] * model.x[i, j, k] for i in L for j in L if i != j)
        <= model.autonomia[k] + sum(model.r[j, k] for j in E)
    )

# (8) Peso máximo permitido en cada municipio
for i in D:
    for k in V:
        model.res.add(model.w[i, k] <= model.peso_max[i])




SolverFactory('glpk').solve(model, tee=True)
model.x.pprint()


solver 'glpk'


ApplicationError: No executable found for solver 'glpk'